<a href="https://colab.research.google.com/github/3S2/Potato-Disease-Classification-CNN-Model/blob/main/Potato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
Lo

2. Mount Google Drive (Optional)


In [ ]:
import os
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
# Base dataset directory
base_dir = "/content/drive/MyDrive/POTATO"  # Replace with your dataset path

# Verify the directory exists
assert os.path.exists(base_dir), f"Dataset not found at {base_dir}"
print(f"Dataset found at {base_dir}")


Dataset found at /content/drive/MyDrive/POTATO


In [ ]:
import os
import shutil
from PIL import Image

# Define dataset paths
base_dir = "/content/drive/MyDrive/POTATO"  # Update to your tomato dataset path
train_dir = os.path.join(base_dir, "train")
validation_dir = os.path.join(base_dir, "validation")

# Verify the dataset exists
if not os.path.exists(base_dir):
    raise FileNotFoundError(f"Dataset not found at {base_dir}")
else:
    print(f"Dataset found at {base_dir}")

# Check and clean images
def verify_images(directory):
    invalid_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                img = Image.open(file_path)
                img.verify()  # Check if image is corrupted
            except Exception as e:
                print(f"Removing corrupted image: {file_path}")
                os.remove(file_path)
                invalid_files.append(file_path)
    return invalid_files

# Verify train and validation sets
print("Verifying training images...")
invalid_train_files = verify_images(train_dir)
print(f"Invalid files in training: {len(invalid_train_files)}")

print("Verifying validation images...")
invalid_val_files = verify_images(validation_dir)
print(f"Invalid files in validation: {len(invalid_val_files)}")


Dataset found at /content/drive/MyDrive/POTATO
Verifying training images...
Invalid files in training: 0
Verifying validation images...
Invalid files in validation: 0


In [ ]:
import os
import random
import shutil

# Define the source and destination paths
base_dir = "/content/drive/MyDrive/"  # Update to your dataset path
dataset_dir = os.path.join(base_dir, "POTATO")
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "validation")
test_dir = os.path.join(base_dir, "test")

# Create train, validation, and test directories
for split_dir in [train_dir, val_dir, test_dir]:
    for class_name in ["early_blight", "late_blight", "healthy"]:
        os.makedirs(os.path.join(split_dir, class_name), exist_ok=True)

# Define split ratios
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1

# Function to split the dataset
def split_dataset(class_name):
    source = os.path.join(dataset_dir, class_name)
    images = os.listdir(source)
    random.shuffle(images)

    train_split = int(len(images) * train_ratio)
    val_split = int(len(images) * (train_ratio + val_ratio))

    for i, img in enumerate(images):
        src_path = os.path.join(source, img)
        if i < train_split:
            dest_path = os.path.join(train_dir, class_name, img)
        elif i < val_split:
            dest_path = os.path.join(val_dir, class_name, img)
        else:
            dest_path = os.path.join(test_dir, class_name, img)

        shutil.copy(src_path, dest_path)

# Apply splitting to each class
for class_name in ["early_blight", "late_blight", "healthy"]:
    split_dataset(class_name)

print("Dataset split into train, validation, and test sets!")


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/POTATO/early_blight'

In [ ]:
import os
from PIL import Image

# Function to verify and summarize dataset structure
def check_dataset_structure(base_path):
    if not os.path.exists(base_path):
        print(f"The dataset path {base_path} does not exist.")
        return None

    dataset_summary = {}
    print(f"Checking dataset structure in: {base_path}\n")
    for folder_name in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder_name)
        if os.path.isdir(folder_path):  # Only process directories
            image_files = [
                file for file in os.listdir(folder_path)
                if os.path.isfile(os.path.join(folder_path, file))
            ]
            num_images = len(image_files)
            dataset_summary[folder_name] = num_images

            # Verify image integrity
            invalid_images = []
            for image_file in image_files:
                image_path = os.path.join(folder_path, image_file)
                try:
                    img = Image.open(image_path)
                    img.verify()  # Check if the image can be opened
                except Exception as e:
                    invalid_images.append(image_file)
                    os.remove(image_path)  # Remove corrupted file
                    print(f"Removed corrupted image: {image_path}")

            print(f"Class '{folder_name}': {num_images - len(invalid_images)} valid images, {len(invalid_images)} invalid images removed.")

    return dataset_summary

# Base dataset directory
base_dir = "/content/drive/MyDrive/OrganizedDataset"  # Update with your dataset path

# Summarize dataset
dataset_summary = check_dataset_structure(base_dir)

# Display the summary
if dataset_summary:
    print("\nDataset Summary:")
    for folder, count in dataset_summary.items():
        print(f"Class '{folder}': {count} images")
else:
    print("\nNo valid dataset structure found.")


Checking dataset structure in: /content/drive/MyDrive/OrganizedDataset

Class 'train': 0 valid images, 0 invalid images removed.
Class 'validation': 0 valid images, 0 invalid images removed.
Class 'test': 0 valid images, 0 invalid images removed.

Dataset Summary:
Class 'train': 0 images
Class 'validation': 0 images
Class 'test': 0 images


In [ ]:
def check_images_in_directory(base_path):
    for root, dirs, files in os.walk(base_path):
        print(f"Checking: {root}")
        print(f" - Subdirectories: {dirs}")
        print(f" - Number of files: {len(files)}\n")

# Check the dataset directory
check_images_in_directory(base_dir)


Checking: /content/drive/MyDrive/OrganizedDataset
 - Subdirectories: ['train', 'validation', 'test', 'augmented']
 - Number of files: 0

Checking: /content/drive/MyDrive/OrganizedDataset/train
 - Subdirectories: ['Potato___Late_blight', 'Potato___Early_blight', 'Potato___healthy']
 - Number of files: 0

Checking: /content/drive/MyDrive/OrganizedDataset/train/Potato___Late_blight
 - Subdirectories: []
 - Number of files: 700

Checking: /content/drive/MyDrive/OrganizedDataset/train/Potato___Early_blight
 - Subdirectories: []
 - Number of files: 700

Checking: /content/drive/MyDrive/OrganizedDataset/train/Potato___healthy
 - Subdirectories: []
 - Number of files: 689

Checking: /content/drive/MyDrive/OrganizedDataset/validation
 - Subdirectories: ['Potato___Late_blight', 'Potato___Early_blight', 'Potato___healthy']
 - Number of files: 0

Checking: /content/drive/MyDrive/OrganizedDataset/validation/Potato___Late_blight
 - Subdirectories: []
 - Number of files: 150

Checking: /content/drive

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from PIL import Image

# Define paths
healthy_dir = "/content/drive/MyDrive/OrganizedDataset/train/Potato___healthy"
augment_dir = "/content/drive/MyDrive/OrganizedDataset/augmented/Potato___healthy"  # Augmented output directory
os.makedirs(augment_dir, exist_ok=True)

# Augmentation setup
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Augment images
def augment_images(input_dir, output_dir, target_count):
    images = os.listdir(input_dir)
    current_count = len(images)
    augmented_count = 0

    for img_name in images:
        if augmented_count >= target_count - current_count:
            break

        img_path = os.path.join(input_dir, img_name)
        img = load_img(img_path)  # Load image
        img_array = img_to_array(img)  # Convert to array
        img_array = img_array.reshape((1,) + img_array.shape)  # Reshape for generator

        # Generate and save augmented images
        for batch in datagen.flow(img_array, batch_size=1, save_to_dir=output_dir,
                                  save_prefix="aug", save_format="jpeg"):
            augmented_count += 1
            if augmented_count >= target_count - current_count:
                break

# Desired count after augmentation
target_image_count = 700  # Same as the other classes in the training set

# Augment images
augment_images(healthy_dir, augment_dir, target_image_count)

print(f"Augmentation complete. Augmented images saved in: {augment_dir}")


Augmentation complete. Augmented images saved in: /content/drive/MyDrive/OrganizedDataset/augmented/Potato___healthy


In [ ]:
import shutil

# Paths
augmented_dir = "/content/drive/MyDrive/OrganizedDataset/augmented/Potato___healthy"
train_healthy_dir = "/content/drive/MyDrive/OrganizedDataset/train/Potato___healthy"

# Move augmented images to the training folder
augmented_images = os.listdir(augmented_dir)
for img_name in augmented_images:
    src_path = os.path.join(augmented_dir, img_name)
    dest_path = os.path.join(train_healthy_dir, img_name)
    shutil.move(src_path, dest_path)

print(f"Moved {len(augmented_images)} augmented images to the training folder.")


Moved 583 augmented images to the training folder.


In [ ]:
import os
from PIL import Image

def remove_corrupted_images(directory):
    corrupted_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                img = Image.open(file_path)
                img.verify()  # Verify if the image is valid
            except Exception as e:
                print(f"Corrupted image found and removed: {file_path}")
                corrupted_files.append(file_path)
                os.remove(file_path)  # Remove the corrupted file
    return corrupted_files

# Dataset directory
base_dir = "/content/drive/MyDrive/OrganizedDataset"  # Update with your dataset path

# Remove corrupted images
corrupted_files = remove_corrupted_images(base_dir)
print(f"\nTotal corrupted images removed: {len(corrupted_files)}")



Total corrupted images removed: 0


In [ ]:
import os
import hashlib

def find_and_remove_duplicates(directory):
    file_hashes = {}
    duplicate_files = []

    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                # Compute hash of the image
                with open(file_path, "rb") as f:
                    file_hash = hashlib.md5(f.read()).hexdigest()

                if file_hash in file_hashes:
                    print(f"Duplicate image found and removed: {file_path}")
                    duplicate_files.append(file_path)
                    os.remove(file_path)  # Remove duplicate
                else:
                    file_hashes[file_hash] = file_path
            except Exception as e:
                print(f"Error processing file: {file_path}, {e}")

    return duplicate_files

# Remove duplicate images
duplicate_files = find_and_remove_duplicates(base_dir)
print(f"\nTotal duplicate images removed: {len(duplicate_files)}")


Duplicate image found and removed: /content/drive/MyDrive/OrganizedDataset/test/Potato___Late_blight/Copy of 0acdc2b2-0dde-4073-8542-6fca275ab974___RS_LB 4857.JPG

Total duplicate images removed: 1


In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models

# Define image dimensions and number of classes
IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS = 224, 224, 3
NUM_CLASSES = 3  # Update based on your dataset

# Load the EfficientNetB0 model without the top layer
efficientnet_base = EfficientNetB0(
    include_top=False,  # Exclude the fully connected layer
    weights='imagenet',  # Pre-trained on ImageNet
    input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)
)

# Freeze the base model's layers
efficientnet_base.trainable = False

# Add custom layers for your dataset
efficientnet_model = models.Sequential([
    efficientnet_base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
efficientnet_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
efficientnet_model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             771 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,378,278 (16.70 MB)

 Trainable params: 328,707 (1.25 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data directories
train_dir = "/content/drive/MyDrive/OrganizedDataset/train"
val_dir = "/content/drive/MyDrive/OrganizedDataset/validation"

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Normalize pixel values
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0/255)

# Load images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=32,
    class_mode='categorical'
)


Found 2089 images belonging to 3 classes.
Found 323 images belonging to 3 classes.


In [ ]:
import tensorflow as tf

# Train EfficientNet model
efficientnet_history = efficientnet_model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
        tf.keras.callbacks.ModelCheckpoint("efficientnet_best.keras", save_best_only=True)
    ]
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


66/66 ━━━━━━━━━━━━━━━━━━━━ 296s 4s/step - accuracy: 0.3040 - loss: 1.1074 - val_accuracy: 0.4644 - val_loss: 1.0599
Epoch 2/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 302s 4s/step - accuracy: 0.3077 - loss: 1.1129 - val_accuracy: 0.0712 - val_loss: 1.1177
Epoch 3/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 272s 4s/step - accuracy: 0.3345 - loss: 1.1065 - val_accuracy: 0.0712 - val_loss: 1.1226
Epoch 4/10
66/66 ━━━━━━━━━━━━━━━━━━━━ 330s 4s/step - accuracy: 0.3435 - loss: 1.1006 - val_accuracy: 0.0712 - val_loss: 1.1071


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)


In [ ]:
efficientnet_base.trainable = True


In [ ]:
# Adjust the learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

# Compile the model with the new optimizer
efficientnet_model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Unfreeze the base model for fine-tuning
efficientnet_base.trainable = True

# Train the model again
efficientnet_history = efficientnet_model.fit(
    train_generator,
    epochs=20,  # Train for more epochs
    validation_data=val_generator,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
        tf.keras.callbacks.ModelCheckpoint("efficientnet_best.keras", save_best_only=True)
    ]
)


Epoch 1/20
49/66 ━━━━━━━━━━━━━━━━━━━━ 3:32 12s/step - accuracy: 0.6391 - loss: 0.7950